# Imports

In [1]:
from scidag import DAG, StartNode, Node
from omegaconf import OmegaConf
dag = DAG()

# CREATING SOME FUNCTIONS

In [2]:
# FIXME 
import math
import numpy as np

def my_sum(x, y) -> float:
    return x + y

def tan(x) -> float:
    return math.tan(x)

class TestClass:
    def __init__(self, test=10) -> None:
        pass

def cos(x, b = TestClass()) -> float:
    return math.cos(x)

class Default:
    def __init__(self, base:int = 2) -> None:
        self.base = base
        
class MyPow:
    def __init__(self, default = Default()) -> None:
        self.base = default.base
        
    def __call__(self, x):
        return pow(x, self.base)

In [3]:
import random
dag.add(StartNode("node_a", random.random))
dag.add(Node("node_b", tan))
dag.add(Node("node_c", MyPow()))
dag.add(Node("node_d", my_sum))

# FIXME
dag.connect("node_a", "node_b", "x")
dag.connect("node_a", "node_c", "x")
dag.connect("node_b", "node_d", "x")
dag.connect("node_c", "node_d", "y")

In [4]:
print(dag.all_edges)

{'node_a': ['node_b.x', 'node_c.x'], 'node_b': ['node_d.x'], 'node_c': ['node_d.y'], 'node_d': []}


In [5]:
# FIXME not working with instantiation
cfg = dag.to_config()
dag = DAG.from_config(cfg)
dag.run("node_b")

HydraZenUnsupportedPrimitiveError: Building: MyPow ..
 The configured value <__main__.Default object at 0x7f5051373750>, for field `default`, is not supported by Hydra -- serializing or instantiating this config would ultimately result in an error.

Consider using `hydra_zen.builds(<class '__main__.Default'>, ...)` create a config for this particular value.

In [ ]:
dag.storage.show()

,target,variable,source,value
0,node_b,x,node_a,0.610939
1,node_c,x,node_a,0.610939
2,node_d,x,node_b,0.700318
3,node_d,y,node_c,0.373247


In [ ]:
print(OmegaConf.to_yaml(dag.to_config()))

info: {}
dag:
  node_a:
    name: node_a
    content:
      _target_: random.random
      _partial_: true
    variables: null
    edges:
    - node_b.x
    - node_c.x
  node_b:
    name: node_b
    content:
      _target_: __main__.tan
      _partial_: true
    variables:
      x:
        type: None
        value: 0.6109392441504008
    edges:
    - node_d.x
  node_c:
    name: node_c
    content:
      _target_: __main__.MyPow
      base: 2
    variables:
      x:
        type: None
        value: 0.6109392441504008
    edges:
    - node_d.y
  node_d:
    name: node_d
    content:
      _target_: __main__.my_sum
      _partial_: true
    variables:
      x:
        type: None
        value: 0.7003178343003321
      'y':
        type: None
        value: 0.373246760043063
    edges: []

